In [1]:
import numpy as np
import torch
import torch.optim as optim
import pickle
import pandas as pd
import sys
import copy

sys.path.append("..\\..\\src")

from datasets import CV2ImageDataset, dataset_loader
from tf_model import Net
#from cifar_model import Net
from model_class import NeuralNet
from run_phase import run_phase
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold
import pickle

from meta_learning_tools import MetaLearn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [2]:

def model_prototype():
    net = Net()
    #nn_model = NeuralNet(net)
    #model_prototype = nn_model.get_model()
    return net#model_prototype

In [3]:
with open('saved_cifar_model_weights_dictionary_with_tl.pkl', 'rb') as fp:
    model_weight_dict = pickle.load(fp)
#print(model_weight_dict) 
model_list = []
for key in model_weight_dict.keys():
    #print(model_weight_dict[key].model)
    model_weight = model_weight_dict[key]
    model = model_prototype()
    model.load_state_dict(model_weight)
    #print(type(model))
    #print(model)
    model_list.append(model.cpu())

print(len(model_list))
#print(model_list)

5


In [4]:
train_df = pd.read_csv('data\\train.csv')
val_df = pd.read_csv('data\\test.csv')

In [5]:

meta_learn = MetaLearn(model_list)
tta_transforms = [
    A.augmentations.transforms.ChannelShuffle(p=0.5),
    A.HorizontalFlip(p=0.5), 
]

train_tta_dict= meta_learn.generate_tta_dict_for_folds(train_df , tta_transforms, device)
val_tta_dict= meta_learn.generate_tta_dict_for_folds(val_df , tta_transforms, device)

tta device:  cuda:0 <class 'torch.Tensor'>
transformed_image.device :  cpu
transformed_image.device :  cpu
Parameter containing:
tensor([[ 0.0232, -0.0050,  0.0293,  ..., -0.0090,  0.0231, -0.0048],
        [ 0.0251, -0.0165,  0.0245,  ...,  0.0028, -0.0187, -0.0047],
        [ 0.0219,  0.0117,  0.0145,  ..., -0.0033,  0.0065, -0.0028],
        ...,
        [-0.0059,  0.0120,  0.0257,  ...,  0.0333,  0.0062,  0.0264],
        [-0.0226, -0.0059,  0.0225,  ...,  0.0040, -0.0121, -0.0185],
        [-0.0142, -0.0259, -0.0037,  ..., -0.0068,  0.0027, -0.0037]],
       device='cuda:0', requires_grad=True)
-1


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument weight in method wrapper___slow_conv2d_forward)

In [ ]:
print(val_tta_dict.keys())#, .keys())

In [ ]:
from datasets import CV2ImageDataset, dataset_loader
from run_phase import run_phase
batch_size = 64
aug = A.Compose([ 
    A.Normalize(),            
    ToTensorV2()])
val_ds = CV2ImageDataset(val_df, transform=aug, device = device)
val_ds_l = dataset_loader(val_ds, batch_size = batch_size)
valloader = val_ds_l.get_dataloader()
# Just Sanity Check
for i, model in enumerate(model_list):
    print('model : ', i)
    val_phase = run_phase(valloader, model.to(device),'Validation', device, loss = torch.nn.CrossEntropyLoss( ))
    val_accuracy_meter, val_loss_meter =  val_phase.run()

In [ ]:
train_tta_df=meta_learn.create_meta_learn_labels_on_dict(train_tta_dict)
val_tta_df=meta_learn.create_meta_learn_labels_on_dict(val_tta_dict)
print( val_tta_df.shape,train_tta_df.shape)

In [ ]:
val_tta_df.head()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

val_acc, val_log_loss = meta_learn.do_meta_learning_on_tta_dicts(RandomForestClassifier(), train_tta_df, val_tta_df, 'custom')

print(val_acc, val_log_loss)